<a href="https://colab.research.google.com/github/MarouenKadri/Algorithmic-Thinking/blob/main/UnetModeltopVersion3855.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))




Found GPU at: /device:GPU:0


In [2]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu May 11 09:47:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W /  70W |    373MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
class DataManager :  
   def __init__(self) :
        self.train_dir1 ="/content/drive/MyDrive/Colab Notebooks/WrinkleDetection_Unet/DatasetV0/image/train"
        self.train_dir2 ="/content/drive/MyDrive/Colab Notebooks/WrinkleDetection_Unet/DatasetV0/texture/train"
        self.test_dir= "/content/drive/MyDrive/Colab Notebooks/WrinkleDetection_Unet/DatasetV0/seg/train"

        self.test_dir1 ="/content/drive/MyDrive/Colab Notebooks/WrinkleDetection_Unet/DatasetV0/image/test"
        self.test_dir2 ="/content/drive/MyDrive/Colab Notebooks/WrinkleDetection_Unet/DatasetV0/texture/test"
        self.load_data()   
   def load_data(self):  
        X_train1=[]
        X_train2=[]
        Y_train=[]
#load Image as rgb 
        image_files1 = [f for f in os.listdir(self.train_dir1) if f.endswith('.jpg') or f.endswith('.png')]
        sorted_files1 = sorted(image_files1)
        for filename in sorted_files1:
              img=cv2.imread(os.path.join(self.train_dir1,filename))
              X_train1.append(img/255.0)
        X_train1=np.array(X_train1)  
#load image as grayscale
        image_files2 = [f for f in os.listdir(self.train_dir2) if f.endswith('.jpg') or f.endswith('.png')]
        sorted_files2 = sorted(image_files2)
        for filename in sorted_files2:
              img=cv2.imread(os.path.join(self.train_dir2,filename),cv2.IMREAD_GRAYSCALE)
              X_train2.append(img/255.0)
        X_train2=np.array(X_train2)
#load image as grayscale
        image_files3 = [f for f in os.listdir(self.test_dir) if f.endswith('.jpg') or f.endswith('.png')]
        sorted_files3 = sorted(image_files3)
        for filename in sorted_files3:
              img=cv2.imread(os.path.join(self.test_dir,filename),cv2.IMREAD_GRAYSCALE)
              Y_train.append(img/255.0)
        Y_train=np.array(Y_train)  
        return X_train1,X_train2,Y_train  
   def load_testData(self): 
        Y_test1=[]
        Y_test2=[]
        image_files1 = [f for f in os.listdir(self.test_dir1) if f.endswith('.jpg') or f.endswith('.png')]
        sorted_files1 = sorted(image_files1)
        for filename in sorted_files1:
              img=cv2.imread(os.path.join(self.test_dir1,filename))
              Y_test1.append(img/255.0)
        Y_test1=np.array(Y_test1)
        image_files2 = [f for f in os.listdir(self.test_dir2) if f.endswith('.jpg') or f.endswith('.png')]
        sorted_files2 = sorted(image_files2)
        for filename in sorted_files2:
             img=cv2.imread(os.path.join(self.test_dir2,filename),cv2.IMREAD_GRAYSCALE)
             Y_test2.append(img/255.0)
        Y_test2=np.array(Y_test2)

        return Y_test1 ,Y_test2
 

     






In [4]:
import os
import cv2
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, \
    Dropout, Lambda,LeakyReLU ,BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import keras.backend as K
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
import math
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split  
import matplotlib.pyplot as plt



device_name = tf.test.gpu_device_name()


################################################################
# Load Data
################################################################


manager=DataManager()
X_train1 ,X_train2,Y_train=manager.load_data()

val_split = 0.1

# Split the data into training and validation sets
X_train1, X_val1, X_train2, X_val2, Y_train, Y_val = train_test_split(
    X_train1, X_train2, Y_train, test_size=val_split, random_state=42)

# Define the image data generator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=0.2,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.01,
    zoom_range=0.01,
    horizontal_flip=True,
    vertical_flip=True,
    # Set the directory to save the augmented images
   
)  
def gen_flow_for_two_inputs(X1, X2, Y, batch_size=4, seed=42):

    gen_X1 = datagen.flow(X1, batch_size=batch_size, seed=seed)
    gen_X2 = datagen.flow(X2, batch_size=batch_size, seed=seed)
    gen_Y = datagen.flow(Y, batch_size=batch_size, seed=seed)
    while True:
        batch_X1 = gen_X1.next()
        batch_X2 = gen_X2.next()  
        batch_Y =gen_Y.next()
        #for i in range(batch_size):
            #plt.subplot(1, 3 * batch_size, i + 1)
            #plt.imshow(batch_X1[i])
            #plt.axis('off')

            #plt.subplot(1, 3 * batch_size, batch_size + i + 1)
            #plt.imshow(batch_X2[i])
            #plt.axis('off')

            #plt.subplot(1, 3 * batch_size, 2 * batch_size + i + 1)
            #plt.imshow(batch_Y[i])
            #plt.axis('off')

        #plt.show()
   
        yield [batch_X1, batch_X2], batch_Y



  
X_train1 = np.reshape(X_train1, (X_train1.shape[0], X_train1.shape[1], X_train1.shape[2], 3))
X_train2 = np.reshape(X_train2, (X_train2.shape[0], X_train2.shape[1], X_train2.shape[2], 1))   
Y_train  = np.reshape(Y_train, (Y_train.shape[0], Y_train.shape[1], Y_train.shape[2], 1)) 


train_data_gen = gen_flow_for_two_inputs(X_train1, X_train2, Y_train, batch_size=4, seed=42)






total_samples = len(X_train1)
batch_size = 4
steps_per_epoch = total_samples // batch_size




checkpoint = ModelCheckpoint(
    filepath='best_w.h5',
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    verbose=1
)
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
lr_max = 0.001
lr_min = 0.0001
epochs = 30
def cosine_annealing(epoch):
    cos_inner = (math.pi * epoch) / epochs
    return lr_min + 0.5 * (lr_max - lr_min) * (1 + math.cos(cos_inner))

def dice_coefficient(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(K.cast(y_true, 'float32'))
    y_pred_f = K.flatten(K.cast(y_pred, 'float32'))
    intersection = K.sum(y_true_f * y_pred_f)
    return (2.0 * intersection + smooth) / (K.sum(y_true_f*y_true_f) + K.sum(y_pred_f*y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coefficient(y_true, y_pred)



def DoubleConv(input_tensor, n_filters, kernel_size=3):
        X = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), padding='same',
                   kernel_initializer='he_normal')(input_tensor)
        X = BatchNormalization()(X)
        X = LeakyReLU(alpha=0.1)(X)
        x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), padding='same',
                   kernel_initializer='he_normal')(X)
        X = BatchNormalization()(X)
        X = LeakyReLU(alpha=0.1)(X)
        return X

def Unet_model() :
    IMG_WIDTH = 640
    IMG_HEIGHT = 640
    IMG_CHANNELS1 = 3
    IMG_CHANNELS2 = 1
    input1 = Input(shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS1))
    input2 = Input(shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS2))
    input1 = Lambda(lambda x: x / 255)(input1)
    input2= Lambda(lambda x: x / 255)(input2)

    input_tensor = concatenate([input1, input2])
    #########################################################################################################################################
    c1=DoubleConv(input_tensor, 64, kernel_size=3)
    p1 = MaxPooling2D((2, 2))(c1)  
    p1 = Dropout(0.1)(p1)
    ############################################################################################################################################
    c2=DoubleConv(p1, 128, kernel_size=3)
    p2 = MaxPooling2D((2, 2))(c2)
    #############################################################################################################################################
    c3 = DoubleConv(p2, 256, kernel_size=3)
    p3=MaxPooling2D ((2,2))(c3)
    p3=Dropout(0.2)(p3)
    ###############################################################################################################################################
    c4 = DoubleConv(p3, 512, kernel_size=3)
    p4 = MaxPooling2D((2, 2))(c4)
    #################################################################################################################################################
    c5 = DoubleConv(p4, 512, kernel_size=3)  

    ################################################################################################################################################
    # Expansive path
    concate1 = Conv2DTranspose(1024, (2, 2), strides=(2, 2), padding='same')(c5)
    concate1 = concatenate([concate1, c4])
    u1 = DoubleConv(concate1, 256, kernel_size=3)
    u1 = Dropout(0.1)(u1)
    #################################################################################################################################################
    # 2nd Upsampling & concatenate
    concate2 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(u1)
    concate2= concatenate([concate2,c3])
    # 3rd Conv2D layer
    u2 = DoubleConv(concate2, 128, kernel_size=3)
    ################################################################################################################################################
    # 4th Upsampling & concatenate
    concate3 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(u2)
    concate3 = concatenate([concate3, c2])
    u3 = DoubleConv(concate3, 64, kernel_size=3)
    u3=Dropout(0.2)(u3)
    ##################################################################################################################################################
    # 5th Upsampling & concatenate
    concate4 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(u3)
    concate4 = concatenate([concate4, c1])
    u4 = DoubleConv(concate4, 64, kernel_size=3)
  
    ##################################################################################################################################################
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(u4)
    model = Model(inputs=[input1,input2], outputs=[outputs])

    return model

if __name__ == "__main__":
    model=Unet_model()
    lr_scheduler = LearningRateScheduler(cosine_annealing)
    model.compile(optimizer='adam', loss=dice_loss, metrics=[dice_coefficient])  
    gen_flow_for_two_inputs(X_train1, X_train2, Y_train, batch_size=4, seed=42)

    #model.summary()
    
    history=model.fit(gen_flow_for_two_inputs(X_train1, X_train2, Y_train, batch_size=3, seed=42),validation_data=([X_val1, X_val2], Y_val),steps_per_epoch=steps_per_epoch,epochs=30,callbacks=[lr_scheduler,checkpoint,early_stopping  ], batch_size=3)
    

Epoch 1/30
60/60 [==============================] - ETA: 0s - loss: 0.6091 - dice_coefficient: 0.3909
Epoch 1: val_loss improved from inf to 0.79631, saving model to best_w.h5
60/60 [==============================] - 95s 1s/step - loss: 0.6091 - dice_coefficient: 0.3909 - val_loss: 0.7963 - val_dice_coefficient: 0.2106 - lr: 0.0010
Epoch 2/30
60/60 [==============================] - ETA: 0s - loss: 0.4172 - dice_coefficient: 0.5828
Epoch 2: val_loss improved from 0.79631 to 0.62083, saving model to best_w.h5
60/60 [==============================] - 69s 1s/step - loss: 0.4172 - dice_coefficient: 0.5828 - val_loss: 0.6208 - val_dice_coefficient: 0.3886 - lr: 9.9753e-04
Epoch 3/30
60/60 [==============================] - ETA: 0s - loss: 0.3534 - dice_coefficient: 0.6466
Epoch 3: val_loss did not improve from 0.62083
60/60 [==============================] - 70s 1s/step - loss: 0.3534 - dice_coefficient: 0.6466 - val_loss: 0.6640 - val_dice_coefficient: 0.3489 - lr: 9.9017e-04
Epoch 4/30
60

In [3]:
import os  
import tensorflow as tf


import cv2
import keras.models
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, \
    Dropout, Lambda,LeakyReLU ,BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import matplotlib.pyplot as plt


import keras

import numpy as np
import keras.backend as K  
import timeit

device_name = tf.test.gpu_device_name()

manage=DataManager()



def dice_coefficient(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(K.cast(y_true, 'float32'))
    y_pred_f = K.flatten(K.cast(y_pred, 'float32'))
    intersection = K.sum(y_true_f * y_pred_f)
    return (2.0 * intersection + smooth) / (K.sum(y_true_f*y_true_f) + K.sum(y_pred_f*y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coefficient(y_true, y_pred)

Y_test1,Y_test2 =manage.load_testData()

#import the model
#model = keras.models.load_model('/content/best_w.h5')
model = keras.models.load_model('/content/best_w.h5',custom_objects={'dice_loss': dice_loss, 'dice_coefficient': dice_coefficient} )
predicted_masks = model.predict([Y_test1,Y_test2],batch_size=4)

threshold = 0.5
predicted_masks = (predicted_masks > threshold).astype('uint8')  

#print(predicted_masks.shape)
for i in range(predicted_masks.shape[0]):
         print(predicted_masks[i]*255)  

         cv2.imwrite('hello{}.png'.format(i), predicted_masks[i]*255)
#######################################################################





15/15 [==============================] - 19s 552ms/step
[[[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 ...

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]]
[[[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 ...

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]]
[[[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 ...

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]]
[[[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 .

In [ ]:
import os  
import tensorflow as tf
import cv2
import keras.models
from keras.models import load_model
from PIL import Image
import numpy as np   
device_name = tf.test.gpu_device_name() 
manage=DataManager()
Y_test1,Y_test2 =manage.load_testData()

test_dir1 ="/content/drive/MyDrive/Colab Notebooks/WrinkleDetection_Unet/DatasetV5/textureMap/test"
image_files1 = [f for f in os.listdir(test_dir1) if f.endswith('.jpg') or f.endswith('.png')]
sorted_files1 = sorted(image_files1)

# Define function to load and save predicted masks with original names
def load_and_save_predicted_masks(model, predicted_images_path):

    # Predict masks for test data using the loaded model
    predicted_masks = model.predict([Y_test1, Y_test2], batch_size=4)

    # Threshold predicted masks
    threshold = 0.5
    predicted_masks = (predicted_masks > threshold).astype('uint8') 
    # Loop through each original image file
    for i, file_name in enumerate(sorted_files1):
        # Load original image
        # Get the predicted mask for the current image
        predicted_mask = predicted_masks[i]
        # Save the predicted mask with the original file name in the predicted images folder
        predicted_mask_path = os.path.join(predicted_images_path, file_name)
        cv2.imwrite(predicted_mask_path, predicted_mask * 255)
        print(f"Predicted mask saved at {predicted_mask_path}")

# Load the trained model
model = load_model('/content/best_w.h5', custom_objects={'dice_loss': dice_loss, 'dice_coefficient': dice_coefficient})
# Define paths to the original images and predicted images folders
predicted_images_path = '/content/predicted_images_path'

# Load and save predicted masks with original names
load_and_save_predicted_masks(model, predicted_images_path)




15/15 [==============================] - 6s 383ms/step
Predicted mask saved at /content/predicted_images_path/ABDNA02_Front Face_T0_STD60_recalee_Segmentation.jpg
Predicted mask saved at /content/predicted_images_path/AGANA_Front Face_T0_STD60_recalee_Segmentation.jpg
Predicted mask saved at /content/predicted_images_path/ALEVA09_Front Face_T0_STD60_recalee_Segmentation.jpg
Predicted mask saved at /content/predicted_images_path/ALIMA03_Front Face_T0_STD60_recalee_Segmentation.jpg
Predicted mask saved at /content/predicted_images_path/ANAMA05_Front Face_T0_STD60_recalee_Segmentation.jpg
Predicted mask saved at /content/predicted_images_path/ANDLI03_Front Face_T0_STD60_recalee_Segmentation.jpg
Predicted mask saved at /content/predicted_images_path/ANGDI09_Front Face_T0_STD60_recalee_Segmentation.jpg
Predicted mask saved at /content/predicted_images_path/ANGGR_Front Face_T0_STD60_recalee_Segmentation.jpg
Predicted mask saved at /content/predicted_images_path/ANGHR04_Front Face_T0_STD60_re